В качестве dataset’а взять Iris, оставив 2 класса:
Iris Versicolor
Iris Virginica

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from scipy.optimize import approx_fprime
import numpy as np
import math

iris = load_iris()
data = np.c_[iris.data, iris.target]

# Оставляем только нужные классы (1 и 2)
data = data[data[:, 4] != 0]

# Меняем класс 2 на класс 0
for i in data:
    if i[4] == 2:
        i[4] = 0

# Определяем эпсилон для вычисления градиента и обхода проблемы деления на ноль       
eps = 1e-12

Реализовать самостоятельно логистическую регрессию

In [2]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def iris_class(x, y):
    l = x[0]*y[0] + x[1]*y[1] + x[2]*y[2] + x[3]*y[3] + x[4]
    return sigmoid(l)

def cost(x):
    cost = 0
    
    for y in data:
        h = iris_class(x, y)
        
        # Если равно 0 или 1, это приведет к вычислению логарифма 0, обходим эту проблему
        if h == 1:
            h -= eps
        elif h ==0:
            h += eps
        
        cost += -y[4] * np.log(h) - (1 - y[4]) * np.log(1 - h)
      
    return cost

Обучить ее методом градиентного спуска

In [3]:
x = np.zeros(5) # Начальные параметры
lr = 0.001      # Скорость обучения

for i in range(25):
    # Вычисляем градиент
    grad = approx_fprime(x, cost, eps)
    
    # Обновляем параметры
    x = x - grad * lr

# Проверяем точность предсказания
errors = 0
for y in data:
    if round(iris_class(x, y)) != y[4]:
        errors +=1

print("Точность предсказания:", 100 * (1 - errors / len(data)), "%")

Точность предсказания: 83.0 %


Методом nesterov momentum

In [4]:
x = np.zeros(5) # Начальные параметры
lr = 0.01       # Скорость обучения
v = 0           # Экспоненциальная скользящая средняя
g = 0.9         # Коэффициент сохранения импульса

for i in range(25):
    # Вычисляем градиент
    grad = approx_fprime(x, cost, eps)
    
    # Обновляем скользящую среднюю
    v = v * g + grad * lr * (1 - g)
    
    # Обновляем параметры, используя скользящую среднюю
    x = x - v

# Проверяем точность предсказания
errors = 0
for y in data:
    if round(iris_class(x, y)) != y[4]:
        errors +=1

print("Точность предсказания:", 100 * (1 - errors / len(data)), "%")

Точность предсказания: 96.0 %


Методом rmsprop

In [5]:
x = np.zeros(5) # Начальные параметры
lr = 0.1      # Скорость обучения
s = np.zeros(5) # Сумма квадратов всех предыдущих градиентов

for i in range(25):
    # Вычисляем градиент
    grad = approx_fprime(x, cost, eps)
   
    # Обновляем сумму квадратов
    s += grad * grad

    # Обновляем параметры, используя корень суммы квадратов
    x = x - grad * lr / np.sqrt(s + eps)
    

# Проверяем точность предсказания
errors = 0
for y in data:
    if round(iris_class(x, y)) != y[4]:
        errors +=1

print("Точность предсказания:", 100 * (1 - errors / len(data)), "%")

Точность предсказания: 89.0 %
